In [2]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time


In [3]:
# 2/28 미니프로젝트 1일차
# 선한영향력 json 데이터 파일받아오기
# 데이터프레임으로 전환후 csv 파일로저장

import json
import requests


def send_api (path, method):
    API_HOST = "https://xn--o39akkz01az4ip7f4xzwoa.com/"
    url = API_HOST + path
    headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
    body = {}

    try:
        if method == 'GET':
            response = requests.get(url, headers=headers)
        elif method == 'POST':
            response = requests.post(url, headers=headers,
                                     data=json.dumps(body, ensure_ascii=False, indent="\t"))
#        print("response status %r" % response.status_code)
#        print("response text %r" % response.text)
    except Exception as ex:
        print(ex)
        
    info = json.loads(response.text)
    store_info=pd.DataFrame(info['storeList'])
    
    return store_info

stores=send_api("/store/search", "POST")
stores.to_csv("C:\/mini_project/stores.csv", index=False)
stores.tail()

,area,item_code,target2_code,addr2,lng,addr1,open_info,target2_etc,type,close_info,zipcode,img_name,name,target1_code,target1_etc,idx,item_etc,lat
1004,경남,99,01,1층 OJY Beauty House,128.897337265659,경남 김해시 김해대로2471번길 8,월~토 오전10~오후8시,,99,매주 일요일,50934,6ad39d97f4a246598a601b0a6092c94b.jpg,O · J Y Beauty House,01,,19,"커트 무료 , 커트제외전시술50%",35.2287008432853
1005,경남,02,"01,02",148호 카페두다,128.684884477157,경남 창원시 성산구 창원대로 888,07:00~21:00,,01,일요일,51532,NaN,카페 두번째다락방 두다,02,,18,,35.2087938761328
1006,인천,99,"01,02",휴먼빌딩 102호 냠냠족발,126.708659384889,인천 남동구 남동대로702번길 25,16시~24시,,01,매주 수요일,21577,c23eb201da984f2584292a85dc145f60.jpg,냠냠족발&보쌈 구월점,01,,17,싱글세트,37.4450764326939
1007,전북,99,"01,02,99",상가 101호,127.111360492773,전북 전주시 완산구 홍산북로 83,09시~18시,소방공무원 50% 할인(공무원증확인),01,"공휴일, 매주 일요일 휴무",54969,c903bb7a61904fa5bbe49277e8debe22.jpg,카페더헤링본,99,아이 본인 및 가족 등 동반인 모두,16,"모든메뉴, 없는메뉴도 만들어드립니다.",35.8183535350013
1008,서울,02,01,1층 EYE OH,126.927395065288,서울 서대문구 신촌로 1,10:00~21:00,,99,없음,03785,NaN,아이오안경원,01,,15,,37.5588739584805


In [4]:
# 3/2 미니프로젝트 3일차
# 데이터 프레임 mysql 에 저장하기

import MySQLdb
from sqlalchemy import create_engine
import pymysql

columns=['area',
         'item_code',
         'target2_code',
         'addr2',
         'lng',
         'addr1',
         'open_info',
         'target2_etc',
         'type',
         'close_info',
         'zipcode',
         'img_name',
         'name',
         'target1_code',
         'target1_etc',
         'idx',
         'item_etc',
         'lat'
]

df= pd.DataFrame(stores, columns=columns)


host = "127.0.0.1"
user = "root"
password = "bit123"
database = "mini_project"

engine = create_engine(f"mysql+mysqldb://{user}:{password}"\
                    f"@{host}:3306/{database}",
                    encoding="utf-8")
conn = engine.connect()

df.to_sql(name="stores_db",
             con=engine,
             if_exists="append",
             index=False)

conn.close()

IntegrityError: (MySQLdb._exceptions.IntegrityError) (1062, "Duplicate entry '1091' for key 'stores_db.PRIMARY'")
[SQL: INSERT INTO stores_db (area, item_code, target2_code, addr2, lng, addr1, open_info, target2_etc, type, close_info, zipcode, img_name, name, target1_code, target1_etc, idx, item_etc, lat) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)]
[parameters: (('인천', '99', '01,02', '103호 (신나는운동회)현일병의까스돈', '126.758130017297', '인천 계양구 당미1길 20', '오전 10시 30분~ 오후 1시 / 오후 5시~10시', '', '01', '매주 일요일', '21012', 'a690c272df9a407c808826e6e297a118.jpeg', '신나는운동회', '01', '', 1095, '돈가스,파스타', '37.5594124939232'), ('서울', '99', '02', '1층 르팽코', '126.96800228252', '서울 용산구 원효로89길 3-14', '12-19', '', '01', '매주 화,수요일', '04316', '0a54a4b994e84272ab644d2001ddb7e7.jpeg', 'LePanko(르팽코)', '01', '', 1094, '아이스커피,우유,오늘의케이크1개', '37.5400642307818'), ('충북', '01', '01', '우미린스테이상가 1층 B동 104호', '127.535461284411', '충북 진천군 덕산읍 대하로 144', '펑일 오전 8시~오후8시', '', '01', '일요일, 공휴일 휴무', '2